## Test the hybrid quasiparticle configuration system

Imports

In [ ]:
from typing import List
import numpy as np
from scipy.sparse import lil_matrix
from NSMFermions.nuclear_physics_utils import SingleParticleState,get_twobody_nuclearshell_model,J2operator,write_j_square_twobody_file
from NSMFermions.hamiltonian_utils import FermiHubbardHamiltonian
from NSMFermions import QuasiParticlesConverterOnlynnpp,QuasiParticlesConverter
from scipy.sparse.linalg import eigsh

/home/ecosta/miniconda3/envs/annealing-nsm-env/lib/python3.10/site-packages/tqdm_joblib/__init__.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
class HybridQuasiParticleConverter():
    
    def __init__(self,):
        pass
    
    def initialize_shell(self,state_encoding:List,fermionic_subset:np.ndarray):
        
        self.fermionic_subset=fermionic_subset
        self.quasiparticle_subset= np.setdiff1d(np.arange(len(state_encoding)), fermionic_subset)
        print(self.quasiparticle_subset)
        #### nn and pp
        couples=[]

        for a,state_a in enumerate(np.array(state_encoding)[self.quasiparticle_subset]):
            for b,state_b in enumerate(np.array(state_encoding)[self.quasiparticle_subset]):
                a_corresponding_index=self.quasiparticle_subset[a]
                b_corresponding_index=self.quasiparticle_subset[b]
                if b_corresponding_index>a_corresponding_index:
                    _,_,ja,ma,_,tza=state_a
                    _,_,jb,mb,_,tzb=state_b
                    if ja==jb and ma==-mb and tza==tzb:
                        couples.append([a_corresponding_index,b_corresponding_index])     

        self.couples=couples



    def new_base_computation(self,base:np.ndarray):
        
        ref_base=base.copy()
        #indices=np.nonzero(base)[0]
        new_base=np.zeros(len(self.couples)+self.fermionic_subset.shape[0])
        value=np.sum(base[self.quasiparticle_subset])
        value2=np.sum(base[self.fermionic_subset])
        
                
        list_of_token_indices=[]
        
        for i in range(len(self.couples)):
            if base[self.couples[i][0]]+base[self.couples[i][1]]!=2 :
                continue
            else:
                new_base[i]+=1
                ref_base[self.couples[i][0]]=0
                ref_base[self.couples[i][1]]=0
        
        new_base[len(self.couples):]=base[self.fermionic_subset]
        if 2*np.sum(new_base[:len(self.couples)]) +np.sum(new_base[len(self.couples):]) == np.sum(base):
            return new_base
        
    def get_the_basis_matrix_transformation(self,basis:np.ndarray):
        
        self.quasiparticle_basis=[]
        self.rest_basis=[]
        
        for i,b in enumerate(basis):
            qp_base=self.new_base_computation(base=b.copy())
            
            if qp_base is not(None):

                self.quasiparticle_basis.append(qp_base)
            else:
                self.rest_basis.append(b.copy())
        self.quasiparticle_basis=np.asarray(self.quasiparticle_basis)
        
        self.rest_basis=np.asarray(self.rest_basis)
        
        self.particles2quasiparticles=lil_matrix((self.quasiparticle_basis.shape[0],basis.shape[0]))
        self.particles2restofstates=lil_matrix((self.rest_basis.shape[0],basis.shape[0]))
        qp_idx=0
        rest_idx=0
        for i,b in enumerate(basis):
            qp_base=self.new_base_computation(base=b.copy())
            
            if qp_base is not(None):
                self.particles2quasiparticles[qp_idx,i]=1.
                qp_idx+=1
            else:
                self.particles2restofstates[rest_idx,i]=1
                rest_idx+=1
                


Load the encoding

In [ ]:
file_name='data/cki'

SPS=SingleParticleState(file_name=file_name)


QPC=QuasiParticlesConverterOnlynnpp()
QPC.initialize_shell(state_encoding=SPS.state_encoding)

QPC.couples.append([1,5])
QPC.couples.append([2,4])

QPC.couples.append([7,11])
QPC.couples.append([8,10])

QPCnp=QuasiParticlesConverter()
QPCnp.initialize_shell(state_encoding=SPS.state_encoding)






print(QPCnp.couples)

[[0, 3], [1, 2], [4, 5], [6, 9], [7, 8], [10, 11], [0, 9], [1, 8], [2, 7], [3, 6], [4, 11], [5, 10]]


#### Test using the basis of the NSM Hamiltonian

In [73]:
size=len(SPS.energies)//2
nparticles_a=2
nparticles_b=2

In [5]:
twobody_matrix,_=get_twobody_nuclearshell_model(file_name=file_name)

Computing the matrix, pls wait... (u_u) 



100%|██████████| 12/12 [00:01<00:00,  7.83it/s]


In [6]:
write_j_square_twobody_file(filename=file_name)

In [101]:
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size,size_b=size,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
NSMHamiltonian.get_external_potential(SPS.energies)
NSMHamiltonian.get_twobody_interaction_optimized(twobody_matrix)
NSMHamiltonian.get_hamiltonian()


J2op=J2operator(size_a=size,size_b=size,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename='data/cki_j2',symmetries=[SPS.total_M_zero])


egs,psigs=NSMHamiltonian.get_spectrum(n_states=10)
print(egs)



Building two-body operator with 1424 terms...


100%|██████████| 1424/1424 [00:00<00:00, 15048.27it/s]


✅ Two-body operator built: shape=(51, 51), nnz=1155
Computing the matrix, pls wait... (u_u) 



100%|██████████| 12/12 [00:00<00:00, 13.14it/s]
592it [00:01, 533.51it/s]


[-30.29539461 -26.88369692 -19.0041005  -15.86699708 -15.31298892
 -14.49860676 -13.61172347 -13.41558684 -12.80630663 -12.62779145]


In [48]:
j_values=[]
for i in range(egs.shape[0]):
    value=J2op.j_value(psigs[:,i])
    print(value,egs[i],i)
    j_values.append(value)
    
    

5.88418203051333e-15 -30.295394613605563 0
2.000000000000004 -26.883696917506825 1
4.000000000000006 -19.00410049996262 2
2.0 -15.866997076428701 3
1.0000000000000018 -15.312988917445386 4
2.0000000000000044 -14.49860675665428 5
3.0000000000000013 -13.611723473962273 6
1.0000000000000007 -13.415586837973553 7
3.0000000000000004 -12.806306628861428 8
1.9999999999999991 -12.627791450539375 9


In [104]:


QPC.get_the_basis_matrix_transformation(NSMHamiltonian.basis.copy())

QPCnp.get_the_basis_matrix_transformation(NSMHamiltonian.basis.copy())


In [ ]:
for base in QPCnp.rest_basis:
    idxs=np.nonzero(base)[0]
    for idx in idxs:
        print(idx)
    print('\n')

In [ ]:
print(QPCnp.rest_basis)

In [ ]:

for i,base in enumerate(QPCnp.quasiparticle_basis):
    idxs=np.nonzero(base)[0]
    for idx in idxs:
        print(QPCnp.couples[idx])
        
    print(np.nonzero(NSMHamiltonian.basis[i]),'\n')
    


In [106]:
print(QPCnp.quasiparticle_basis.shape)
print(QPC.quasiparticle_basis.shape)

(21, 12)
(25, 10)


In [107]:

print(QPC.quasiparticle_basis.shape)
hamiltonian_Q=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
egsQ,psi_Q=eigsh(hamiltonian_Q,k=10,which='SA')

hamiltonian_Qnp=QPCnp.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPCnp.particles2quasiparticles.T
egsQnp,psi_Qnp=eigsh(hamiltonian_Qnp,k=10,which='SA')

print(egsQnp,egsQ)


psi_Q2particle=QPC.particles2quasiparticles.T @ psi_Q[:,0]
psi_Q2particle/=np.linalg.norm(psi_Q2particle)

psi_Qnp2particle=QPCnp.particles2quasiparticles.T @ psi_Qnp[:,0]
psi_Qnp2particle/=np.linalg.norm(psi_Qnp2particle)

print('\n')

#print(psigs[:,0].dot(psi_Q2particle.conj())*np.conjugate(psigs[:,0].dot(psi_Q2particle.conj())))
fs=[]
for i in range(egs.shape[0]):
    print(psigs[:,i].dot(psi_Qnp2particle.conj())*np.conjugate(psigs[:,i].dot(psi_Qnp2particle.conj())),j_values[i])
    fs.append(psigs[:,i].dot(psi_Qnp2particle.conj())*np.conjugate(psigs[:,i].dot(psi_Qnp2particle.conj())))
print('\n')
print(np.sum(fs),'\n')
fs=[]
for i in range(egs.shape[0]):
    print(psigs[:,i].dot(psi_Q2particle.conj())*np.conjugate(psigs[:,i].dot(psi_Q2particle.conj())),j_values[i])
    fs.append(psigs[:,i].dot(psi_Q2particle.conj())*np.conjugate(psigs[:,i].dot(psi_Q2particle.conj())))
    
print(np.sum(fs))


(25, 10)
[-24.63148876 -19.48711395 -14.10273567 -12.52467658 -11.59318595
 -10.98143642  -9.96412612  -8.65366588  -8.19065533  -5.76565536] [-26.15701513 -20.67582042 -13.96532396 -13.89674989 -13.24246285
 -11.12096904 -10.77087133  -9.97957795  -9.57765461  -8.68177795]


0.7557518116625264 5.88418203051333e-15
0.0011184538724464492 2.000000000000004
6.489244527185443e-07 4.000000000000006
5.8711917572999455e-05 2.0
1.6116367021401446e-33 1.0000000000000018
3.4223223939738185e-32 2.0000000000000044
2.4388324281240583e-32 3.0000000000000013
9.645224027056053e-33 1.0000000000000007
2.830252800694554e-31 3.0000000000000004
0.0016128443104605282 1.9999999999999991


0.7585424706874591 

0.28308677513474284 5.88418203051333e-15
0.5352666872843793 2.000000000000004
0.16039662796609266 4.000000000000006
2.0382626016063273e-05 2.0
1.428914372994944e-33 1.0000000000000018
1.6491265300107052e-32 2.0000000000000044
1.94141790220073e-31 3.0000000000000013
3.781734184884995e-33 1.00000000000000

#### Search of the best set of fermionic DOF

## Random Pair decomposition

Imports

In [53]:
from typing import List
import numpy as np
from scipy.sparse import lil_matrix
from NSMFermions.nuclear_physics_utils import SingleParticleState,get_twobody_nuclearshell_model
from NSMFermions.hamiltonian_utils import FermiHubbardHamiltonian
from NSMFermions import QuasiParticlesConverterOnlynnpp,QuasiParticlesConverter
from scipy.sparse.linalg import eigsh

Quasiparticle class

In [54]:
file_name='data/cki'

SPS=SingleParticleState(file_name=file_name)



In [55]:
negative_m_states=[]
for i in range(len(SPS.state_encoding)):
    n,l,j,m,t,tz=SPS.state_encoding[i]
    if m<0:
        negative_m_states.append(i)

twin_states={}
for p in negative_m_states:
    n,l,j,m,t,tz=SPS.state_encoding[p]
    twin_states[p]=[]
    for q in range(len(SPS.state_encoding)):
        n2,l2,j2,m2,t2,tz2=SPS.state_encoding[q]
        if  m2==-m :
            twin_states[p].append(q)
            
print(twin_states)

{0: [3, 9], 1: [2, 5, 8, 11], 4: [2, 5, 8, 11], 6: [3, 9], 7: [2, 5, 8, 11], 10: [2, 5, 8, 11]}


Start a random decomposition

In [ ]:
new_couples=[]
for n in negative_m_states:
    n_twin=np.random.choice(twin_states[n])
    
    new_couples.append([n,n_twin])
    for ts in twin_states.values():
        if n_twin in ts:
            ts.remove(n_twin)
        

In [ ]:
print(new_couples)

In [ ]:

QPC=QuasiParticlesConverterOnlynnpp()
QPC.couples=new_couples
print(QPC.couples)

#### Test using the basis of the NSM Hamiltonian

In [ ]:
size=len(SPS.energies)//2
nparticles_a=2
nparticles_b=2

In [ ]:
twobody_matrix,_=get_twobody_nuclearshell_model(file_name=file_name)

In [ ]:
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size,size_b=size,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
NSMHamiltonian.get_external_potential(SPS.energies)
NSMHamiltonian.get_twobody_interaction_optimized(twobody_matrix)
NSMHamiltonian.get_hamiltonian()

egs,psigs=NSMHamiltonian.get_spectrum(n_states=1)
print(egs)



In [ ]:

QPC.get_the_basis_matrix_transformation(NSMHamiltonian.basis.copy())


In [ ]:
print(QPC.particles2quasiparticles.shape)
print(QPC.quasiparticle_basis.shape)

In [ ]:
hamiltonian_Q=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
egsQ,psi_Q=eigsh(hamiltonian_Q,k=1,which='SA')

print(egsQ)

psi_Q2particle=QPC.particles2quasiparticles.T @ psi_Q[:,0]
psi_Q2particle/=np.linalg.norm(psi_Q2particle)


print(psigs[:,0].dot(psi_Q2particle.conj())*np.conjugate(psigs[:,0].dot(psi_Q2particle.conj())))



In [ ]:
print(hamiltonian_Q)

#### Get a variational algorithm

In [ ]:
epochs=3000
min=1000
for i in range(epochs):
    negative_m_states=[]
    for r in range(len(SPS.state_encoding)):
        n,l,j,m,t,tz=SPS.state_encoding[r]
        if m<0:
            negative_m_states.append(r)

    twin_states={}
    for p in negative_m_states:
        n,l,j,m,t,tz=SPS.state_encoding[p]
        twin_states[p]=[]
        for q in range(len(SPS.state_encoding)):
            n2,l2,j2,m2,t2,tz2=SPS.state_encoding[q]
            if  m2==-m :
                twin_states[p].append(q)
    new_couples=[]
    for n in negative_m_states:
        n_twin=np.random.choice(twin_states[n])
        
        new_couples.append([n,n_twin])
        for ts in twin_states.values():
            if n_twin in ts:
                ts.remove(n_twin)
    QPC.couples=new_couples
    QPC.get_the_basis_matrix_transformation(NSMHamiltonian.basis.copy())
    hamiltonian_Q=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
    egsQ,psi_Q=eigsh(hamiltonian_Q,k=1,which='SA')

    
    psi_Q2particle=QPC.particles2quasiparticles.T @ psi_Q[:,0]
    psi_Q2particle/=np.linalg.norm(psi_Q2particle)


    if egsQ<min:
        print(psigs[:,0].dot(psi_Q2particle.conj())*np.conjugate(psigs[:,0].dot(psi_Q2particle.conj())))

        print(egsQ)
        min=egsQ
        best_couples=new_couples
    

In [ ]:
print(best_couples)

In [ ]:
for i,b in enumerate(QPC.quasiparticle_basis):
    couple_idxs=(np.nonzero(b)[0])
    for couple_idx in couple_idxs:
        print(QPC.couples[couple_idx])
    print(psi_Q[i]*np.conj(psi_Q[i]))
    print('\n')

In [ ]:
import matplotlib.pyplot as plt

#plt.bar(np.arange(psigs.shape[0])+0.5,psi_Q2particle*psi_Q2particle.conj())
plt.scatter(np.arange(psigs.shape[0]),psi_HQ2particle*psi_HQ2particle.conj()-psigs[:,0]*psigs[:,0].conj())
plt.show()

## Check the results using the neutron proton in the SD shell

In [1]:
from NSMFermions.hamiltonian_utils import FermiHubbardHamiltonian
from NSMFermions.nuclear_physics_utils import get_twobody_nuclearshell_model,SingleParticleState,J2operator
import numpy as np
import torch
from typing import Dict
import scipy
from NSMFermions.qml_models import AdaptVQEFermiHubbard
from NSMFermions.qml_utils.train import Fit
from NSMFermions.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt
from NSMFermions.utils_quasiparticle_approximation import QuasiParticlesConverterOnlynnpp,QuasiParticlesConverter
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import eigsh

file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)
twobody_matrix,_=get_twobody_nuclearshell_model(file_name=file_name)
print(SPS.energies.shape)


nparts=[(0,2),(0,4),(0,6),(0,8),(0,10),(2,2),(2,4),(2,6),(2,8),(2,10),(4,4),(4,6),(4,8),(4,10),(6,6),(6,8),(6,10),(8,8),(8,10),(10,10)]
titles=[r'$^{18}$O',r'$^{20}$O',r'$^{22}$O',r'$^{24}$O',r'$^{26}$O',r'$^{20}$Ne',r'$^{22}$Ne',r'$^{24}$Ne',r'$^{26}$Ne',r'$^{28}$Ne',r'$^{24}$Mg',r'$^{26}$Mg',r'$^{28}$Mg',r'$^{30}$Mg',r'$^{28}$Si',r'$^{30}$Si',r'$^{32}$Si',r'$^{32}$S',r'$^{34}$S',r'$^{36}$Ar']

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{18}$O'

/home/ecosta/miniconda3/envs/annealing-nsm-env/lib/python3.10/site-packages/tqdm_joblib/__init__.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Computing the matrix, pls wait... (u_u) 



100%|██████████| 24/24 [00:45<00:00,  1.88s/it]

(24,)


In [ ]:
# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator
energy_errors=[]
abs_energy_errors=[]
fidelities=[]
for idx,npart in enumerate(nparts):
    nparticles_a,nparticles_b=npart
    title=titles[idx]
    
    # compute the NSM Hamiltonian
    NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
    print('size=',size_a+size_b,size_b)
    NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
    print(NSMHamiltonian.external_potential.shape)
    NSMHamiltonian.get_twobody_interaction_optimized(twobody_dict=twobody_matrix)
    NSMHamiltonian.get_hamiltonian()

    gpu_hamiltonian=(NSMHamiltonian.hamiltonian)
    egs,psi0=eigsh(gpu_hamiltonian,k=1,which='SA')

    print(egs)
    QPC=QuasiParticlesConverter()

    QPC.initialize_shell(state_encoding=SPS.state_encoding)


    #just for the basis
    QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)
    
    hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
    
    gpu_hamiltonian_qq=(hamiltonian_qq)
    egs_qq,psi_q=eigsh(gpu_hamiltonian_qq,k=1,which='SA')
    
    psi_q=QPC.particles2quasiparticles.T @ psi_q[:,0]
    delta=np.abs((egs_qq[0]-egs[0])/egs[0])
    energy_errors.append(delta)
    abs_energy_errors.append(np.abs(egs_qq[0]-egs[0]))
    
    fidelity=(psi_q[:].dot(psi0[:,0])).conjugate()*(psi_q[:].dot(psi0[:,0]))
    fidelities.append(1-fidelity.item())
    